In [64]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS, Chroma
from langchain_openai import OpenAIEmbeddings
import os
import pickle
import hashlib
import warnings
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer
import glob
from pathlib import Path
from tqdm import tqdm
from langchain.embeddings.base import Embeddings

In [7]:
path = Path(r"C:\Users\Shaaf\Desktop\Data Science\Practice Projects\PDF_Video Reader\PDF_Samples")

In [48]:
pdf_files=list(path.rglob("**.pdf"))

In [49]:
pdf_files

[WindowsPath('C:/Users/Shaaf/Desktop/Data Science/Practice Projects/PDF_Video Reader/PDF_Samples/414.pdf'),
 WindowsPath('C:/Users/Shaaf/Desktop/Data Science/Practice Projects/PDF_Video Reader/PDF_Samples/A Handbook of statistical distributions Krishmoorthi.pdf'),
 WindowsPath('C:/Users/Shaaf/Desktop/Data Science/Practice Projects/PDF_Video Reader/PDF_Samples/Analysis_of_Time_Series_An_Introduction.pdf'),
 WindowsPath('C:/Users/Shaaf/Desktop/Data Science/Practice Projects/PDF_Video Reader/PDF_Samples/Approved_Document_B_volume_1_Dwellings_2019_edition_incorporating_2020_2022_and_2025_amendments_collated_with_2026_and_2029_amendments.pdf'),
 WindowsPath('C:/Users/Shaaf/Desktop/Data Science/Practice Projects/PDF_Video Reader/PDF_Samples/ISO19650-2Edition4.pdf')]

In [ ]:
###creating hash for the documents

def folder_fingerprint(folder_path):
    hasher = hashlib.sha256()

    for root, _, files in os.walk(folder_path):
        for file in sorted(files):
            file_path = os.path.join(root, file)
            stat = os.stat(file_path)
            hasher.update(file.encode())
            hasher.update(str(stat.st_size).encode())
            hasher.update(str(stat.st_mtime).encode())

    return hasher.hexdigest()


In [51]:
import json
folder_path = path
FINGERPRINT_FILE = "folder_fingerprint.json"

def load_old_fingerprint():
    if os.path.exists(FINGERPRINT_FILE):
        with open(FINGERPRINT_FILE, "r") as f:
            return json.load(f).get("fingerprint")
    return None

def save_fingerprint(fp):
    with open(FINGERPRINT_FILE, "w") as f:
        json.dump({"fingerprint": fp}, f)

In [ ]:
DOC_CACHE_FILE = "cached_documents.pkl"

new_folder_fingerprint = folder_fingerprint(folder_path)
old_folder_fingerprint = load_old_fingerprint()

documents = []

cache_exists = os.path.exists(DOC_CACHE_FILE)

if new_folder_fingerprint != old_folder_fingerprint or not cache_exists:
    print("📂 Reloading documents (folder changed or cache missing)...")

    for pdf in tqdm(pdf_files, desc="Loading PDFs"):
        try:
            loader = PyPDFLoader(str(pdf))
            docs = loader.load()
            documents.extend(docs)
        except Exception as e:
            print(f"❌ Error loading {pdf}: {e}")

    # Save cache
    with open(DOC_CACHE_FILE, "wb") as f:
        pickle.dump(documents, f)

    save_fingerprint(new_folder_fingerprint)

else:
    print("✅ Folder unchanged. Using cached documents.")

    with open(DOC_CACHE_FILE, "rb") as f:
        documents = pickle.load(f)


📂 Reloading documents (folder changed or cache missing)...


Loading PDFs: 100%|██████████| 5/5 [00:31<00:00,  6.33s/it]


In [59]:
print(len(documents))
print(documents[1].page_content[:500])
print(documents[1].metadata)


1049
2023 National Electric Code (NEC®), OVERVIEW 
A SunCam Online Continuing Education Course 
 
 
www.SunCam.com  Copyright© 2023 John A Camara Page 2 of 42 
 
      HISTORY and CODE OVERVIEW1 
 
Edison invented the first practical incandescent light bulb in 1879. In the very same year, the 
National Association of Fire Engineers met for the purpose of establishing requirements for 
electrical installations. As with many standards, in a few years there were six different standards 
in place. Theref
{'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2023-11-02T13:12:38-07:00', 'author': 'Bill Dunn', 'moddate': '2023-11-02T13:12:38-07:00', 'source': 'C:\\Users\\Shaaf\\Desktop\\Data Science\\Practice Projects\\PDF_Video Reader\\PDF_Samples\\414.pdf', 'total_pages': 42, 'page': 1, 'page_label': '2'}


In [60]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [61]:
chunked_document = text_splitter.split_documents(documents=documents)

In [66]:

class HFEmbeddings(Embeddings):
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        return self.model.encode(
            texts,
            normalize_embeddings=True,
            show_progress_bar=False
        ).tolist()

    def embed_query(self, text):
        return self.model.encode(
            text,
            normalize_embeddings=True
        ).tolist()

In [68]:
from sentence_transformers import SentenceTransformer

qwen_model = SentenceTransformer(
    "all-MiniLM-L6-v2",
        trust_remote_code=True
)

if not os.path.exists('faiss_qwen'):
    os.makedirs('faiss_qwen')
    print("Creating new FAISS vectorstore...")
    embeddings = HFEmbeddings(qwen_model)

    vectorstore = FAISS.from_documents(
        documents=chunked_document,
        embedding=embeddings)
    vectorstore.save_local("faiss_qwen")

else:
    print("Loading existing FAISS vectorstore...")
    embeddings = HFEmbeddings(qwen_model)
    vectorstore = FAISS.load_local(
    "faiss_qwen",
    embeddings,
    allow_dangerous_deserialization=True
    )

In [69]:
vectorstore

In [163]:
query = "what is time series analysis?"

docs = vectorstore.similarity_search(
    query=query,
    k=10
)

In [148]:
docs

[Document(id='94914cda-c3d2-45d6-b266-025e4bbd5e33', metadata={'producer': 'Adobe Acrobat Pro 11.0.20 Paper Capture Plug-in', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-08-18T01:21:09+03:00', 'author': 'dbsubasi', 'moddate': '2017-12-23T12:10:40+05:00', 'title': 'DjVu Document', 'source': 'C:\\Users\\Shaaf\\Desktop\\Data Science\\Practice Projects\\PDF_Video Reader\\PDF_Samples\\Analysis_of_Time_Series_An_Introduction.pdf', 'total_pages': 293, 'page': 14, 'page_label': '15'}, page_content="deterministic. But most time series are stochastic in that the future is only \npartly determined by past values, so that exact predictions are impossible and \nmust be replaced by the idea that future values have a probability distribution \nwhich is conditioned by a knowledge of past values. \n1.3 OBJECTIVES OF TIME-SERIES ANALYSIS \nThere are several possible objectives in analysing a time series. These \nobjectives may be classified as description, explanation, prediction and \n

In [164]:
context = "\n\n".join(
    [
        f"(Page {d.metadata.get('page','?')}) {d.page_content}"
        for d in docs
    ]
)

In [150]:
context

"(Page 14) deterministic. But most time series are stochastic in that the future is only \npartly determined by past values, so that exact predictions are impossible and \nmust be replaced by the idea that future values have a probability distribution \nwhich is conditioned by a knowledge of past values. \n1.3 OBJECTIVES OF TIME-SERIES ANALYSIS \nThere are several possible objectives in analysing a time series. These \nobjectives may be classified as description, explanation, prediction and \ncontrol, and will be considered in turn. \n(a) Description \nWhen presented with a time series, the first step in the analysis is usually to plot \nthe data and to obtain simple descriptive measures of the main properties of \nthe series as described in Chapter 2. For example, looking at Figure 1.3 it can \nbe seen that there is a regular seasonal effect, with sales 'high' in winter and \n'low' in summer It also looks as though annual sales are increasing (i e show\n\n(Page 14) Objectives of time-

In [173]:
prompt = f"""
You are a policy compliance expert.

Answer the question using ONLY the information provided below.
If the answer is not present in the text, respond with:
"Not specified in the document."

Context:
{context_list}

Question:
{query}
"""


In [135]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama3.1:8b",
    temperature=0.2
)


In [136]:
response = llm.invoke(prompt)
print(response)

Time series analysis is the process of analyzing data that has been collected over a period of time, typically in equal intervals, to identify patterns, trends, and correlations within the data. It involves using statistical techniques to understand how the data behaves over time and to make predictions about future values.

According to the text (Page 14), "Much statistical theory is concerned with random samples of independent observations. The special feature of time-series analysis is the fact that successive observations are usually not independent and that the analysis must take into account the time order of the observations."

The objectives of time series analysis, as mentioned in Chapter 1.3 (Page 14), include description, explanation, prediction, and control.

Time series analysis can be used to identify trends, seasonal fluctuations, and correlations within the data, as well as to make predictions about future values. It is a complex process that requires careful considerat

In [152]:
from sentence_transformers import CrossEncoder

In [153]:
reranker = CrossEncoder("BAAI/bge-reranker-base")

In [154]:
pairs = [(query, doc.page_content) for doc in docs]

In [155]:
pairs

[('what is time series analysis?',
  "deterministic. But most time series are stochastic in that the future is only \npartly determined by past values, so that exact predictions are impossible and \nmust be replaced by the idea that future values have a probability distribution \nwhich is conditioned by a knowledge of past values. \n1.3 OBJECTIVES OF TIME-SERIES ANALYSIS \nThere are several possible objectives in analysing a time series. These \nobjectives may be classified as description, explanation, prediction and \ncontrol, and will be considered in turn. \n(a) Description \nWhen presented with a time series, the first step in the analysis is usually to plot \nthe data and to obtain simple descriptive measures of the main properties of \nthe series as described in Chapter 2. For example, looking at Figure 1.3 it can \nbe seen that there is a regular seasonal effect, with sales 'high' in winter and \n'low' in summer It also looks as though annual sales are increasing (i e show"),
 (

In [156]:
scores = reranker.predict(pairs)

In [165]:
# 4. Sort by score
reranked_docs = sorted(
    zip(docs, scores),
    key=lambda x: x[1],
    reverse=True
)

In [166]:
page_contents = [doc.page_content for doc, score in reranked_docs]

In [167]:
reranked_docs = [doc for doc, score in reranked_docs]

In [168]:
reranked_docs

[Document(id='94914cda-c3d2-45d6-b266-025e4bbd5e33', metadata={'producer': 'Adobe Acrobat Pro 11.0.20 Paper Capture Plug-in', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-08-18T01:21:09+03:00', 'author': 'dbsubasi', 'moddate': '2017-12-23T12:10:40+05:00', 'title': 'DjVu Document', 'source': 'C:\\Users\\Shaaf\\Desktop\\Data Science\\Practice Projects\\PDF_Video Reader\\PDF_Samples\\Analysis_of_Time_Series_An_Introduction.pdf', 'total_pages': 293, 'page': 14, 'page_label': '15'}, page_content="deterministic. But most time series are stochastic in that the future is only \npartly determined by past values, so that exact predictions are impossible and \nmust be replaced by the idea that future values have a probability distribution \nwhich is conditioned by a knowledge of past values. \n1.3 OBJECTIVES OF TIME-SERIES ANALYSIS \nThere are several possible objectives in analysing a time series. These \nobjectives may be classified as description, explanation, prediction and \n

In [169]:
context_list = [doc.page_content for doc in reranked_docs]

In [171]:
context_list = "\n\n".join(context_list)

In [172]:
context_list

"deterministic. But most time series are stochastic in that the future is only \npartly determined by past values, so that exact predictions are impossible and \nmust be replaced by the idea that future values have a probability distribution \nwhich is conditioned by a knowledge of past values. \n1.3 OBJECTIVES OF TIME-SERIES ANALYSIS \nThere are several possible objectives in analysing a time series. These \nobjectives may be classified as description, explanation, prediction and \ncontrol, and will be considered in turn. \n(a) Description \nWhen presented with a time series, the first step in the analysis is usually to plot \nthe data and to obtain simple descriptive measures of the main properties of \nthe series as described in Chapter 2. For example, looking at Figure 1.3 it can \nbe seen that there is a regular seasonal effect, with sales 'high' in winter and \n'low' in summer It also looks as though annual sales are increasing (i e show\n\nwhen a variable does not have an instan

In [174]:
response = llm.invoke(prompt)
print(response)

Time-series analysis is the process of analyzing data that varies over time, with the goal of understanding and predicting future values. It involves examining patterns, trends, and relationships in the data, often using statistical techniques such as autocorrelation functions and spectral analysis.

The objectives of time-series analysis include:

1. Description: Understanding the main properties of the series.
2. Explanation: Identifying the underlying factors that influence the series.
3. Prediction: Forecasting future values based on past patterns.
4. Control: Using the analysis to inform decision-making and control processes.

Time-series analysis can be challenging due to the correlated nature of the data, which is different from classical statistical inference that assumes independent observations.


In [176]:
####function for retriever
def retriever(query:str,k: int=10):
    docs = vectorstore.similarity_search(query=query, k=k)
    return "\n\n".join([doc.page_content for doc in docs])

### function for rereanker
from sentence_transformers import CrossEncoder
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2") 

def reranker(query, docs):
    pairs = [(query, doc.page_content) for doc in docs]
    scores = reranker.predict(pairs)
    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [177]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama3.1",
    temperature=0.1
)

In [ ]:
SYSTEM_PROMPT = """
You are an intelligent retrieval agent.

Rules:
- Decide when to retrieve documents
- Rewrite the query if needed
- Retrieve again if context is insufficient
- Answer ONLY using retrieved content
- Say "I don't know" if evidence is missing
"""

In [ ]:
##querry re-wrting and retrieval function
def rewrite_query(question, context):
    prompt = f"""
    You are a query rewriting assistant.

    Original question:
    {question}

    Existing context (may be empty):
    {context}

    Rewrite the question into a concise, precise search query
    that would work well for semantic document retrieval.
    Do NOT answer the question.
    Return ONLY the rewritten query.
    """

    rewritten = llm.invoke(prompt).strip()
    return rewritten


In [ ]:
##querry re-wrting and retrieval function
def agentic_rag(question):
    state = {
        "question": question,
        "context": "",
        "attempts": 0
    }

    while state["attempts"] < 3:
        state["attempts"] += 1

        # Step 1: Rewrite query
        search_query = rewrite_query(
            question=state["question"],
            context=state["context"]
        )

        # Step 2: Retrieve
        retrieved_text = retrieve(search_query)

        # Step 3: Accumulate context
        state["context"] += "\n\n" + retrieved_text

        # Step 4: Ask LLM if answer is possible
        prompt = f"""
        Question:
        {state['question']}

        Context:
        {state['context']}

        If the context is sufficient, answer the question.
        Otherwise, respond ONLY with:
        NEED_MORE_INFO
        """

        response = llm.invoke(prompt)

        if "NEED_MORE_INFO" not in response:
            return response

    return "I don't have enough information to answer this question."
